# Example case of estimating observing time for assumed A10 clusters #

This thread shows how to use _M2_ProposalTools_ to estimate the required time to observe clusters based on a single pointing.

In [1]:
import M2_ProposalTools.WorkHorse as WH
import numpy as np
import astropy.units as u
import M2_ProposalTools.FilterImages as FI
import scipy

Let's just take the case of a few masses at high redshift ($z=1.0$)

In [2]:
M500s = np.array([3.0,4.0,5.0,6.0])*1e14*u.M_sun
zs    = np.array([1.0,1.0,1.0,1.0]) 

In [3]:
h70            = 1                          # Normalization of Hubble parameter
fwhm           = 10.0                       # Roughly the resolution of MUSTANG-2
pixsize        = 2.0                        # Assume this pixel size
s2f            = np.sqrt(8.0*np.log(2.0))   # Conversion between FWHM and sigma
pix_sigma      = fwhm/(pixsize*s2f)         # Gaussian sigma, in pixel size
scansize       = 3.0                        # Assume scan sizes of 3 arcminutes
y2k            = -3.3                       # Approximate conversion.

We'll just dive into some of the functions and get what we want: 

In [4]:
TotalTime=0
for M500,z in zip(M500s,zs):

    y500       = WH.y_delta_from_mdelta(M500.value,z,delta=500,YMrel="A10",ycyl=False,h70=h70)
    
    ymap       = WH.make_A10Map(M500,z,pixsize=pixsize,Dist=True)
    mymap      = WH.smooth_by_M2_beam(ymap,pixsize=pixsize)
    
    tab        = WH.get_xfertab(scansize)
    yxfer_init = FI.apply_xfer(mymap,tab,pixsize)
    yxfer      = scipy.ndimage.filters.gaussian_filter(yxfer_init, pix_sigma*0.9)
    uKmap      = yxfer*y2k*1e6
    uJymap     = uKmap * (56/73.0)
    MS         = 56.2
    
    #print("pixel size: ",pixsize)
    SZpeak_uK  = np.min(uKmap)
    SZpeak_uJy = np.min(uJymap)
    SZpeak_yr  = np.max(ymap)
    SZpeak_yc  = np.max(mymap)
    SZpeak_yx  = np.max(yxfer)
    ObsTime    = (MS*10/SZpeak_uJy)**2
    TelTime    = np.ceil(ObsTime*8)/4.0
    print("M500 (1e14): ",(M500/1e14).value)
    print("Redshift: ",z)
    #print("R500 (arcmin): ",Theta500*60*180/np.pi)
    print("Y500 (1e12 sr): ",y500*1e12)
    print("Unfiltered y peak: ", SZpeak_yr)
    print("Beam-convolved y peak: ", SZpeak_yc)
    print("Filtered y peak: ", SZpeak_yx)
    print("Filtered peak, uK: ", SZpeak_uK)
    print("Filtered peak, uJy/beam: ", SZpeak_uJy)
    print("Time to 10 sigma peak detection (hrs): ",ObsTime)
    print("================================================")
    TotalTime += TelTime
    
print("Total Telescope Time: ", TotalTime)

M500 (1e14):  3.0
Redshift:  1.0
Y500 (1e12 sr):  9.607250432078944
Unfiltered y peak:  9.619897496889924e-05
Beam-convolved y peak:  8.860120011199452e-05
Filtered y peak:  4.712003987027812e-05
Filtered peak, uK:  -155.4961315719178
Filtered peak, uJy/beam:  -119.28470367160817
Time to 10 sigma peak detection (hrs):  22.19745167103094
M500 (1e14):  4.0
Redshift:  1.0
Y500 (1e12 sr):  16.032087067090995
Unfiltered y peak:  0.00013376973214360876
Beam-convolved y peak:  0.00012461706064739533
Filtered y peak:  6.38786680129082e-05
Filtered peak, uK:  -210.79960444259706
Filtered peak, uJy/beam:  -161.70928559980047
Time to 10 sigma peak detection (hrs):  12.078213834594965
M500 (1e14):  5.0
Redshift:  1.0
Y500 (1e12 sr):  23.850082514564303
Unfiltered y peak:  0.00017266723341949052
Beam-convolved y peak:  0.0001609109038327221
Filtered y peak:  7.946347504129425e-05
Filtered peak, uK:  -262.229467636271
Filtered peak, uJy/beam:  -201.16233133741338
Time to 10 sigma peak detection (hrs